# LSTM

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# ind_data = pd.read_csv('../data/raw_data/All_indicator_1d.csv',index_col= 0)
# price_data = pd.read_csv('../data/data_indicators/BTC_data_1d_2019-01-01.csv',index_col= 0)

# # colab
ind_data = pd.read_csv('All_indicator_1d.csv',index_col= 0)
price_data = pd.read_csv('BTC_data_1d_2019-01-01.csv',index_col= 0)

from ds_toolbox import dp_mv_DropCols
ind_data = dp_mv_DropCols(ind_data,0.5)

price_data['date'] = pd.to_datetime(price_data['Open time']).dt.strftime('%Y-%m-%d')
data = ind_data.merge(price_data,how = 'left', on='date')

from ts_toolbox import plot_multiTS
# fig = plot_multiTS(data, [0, 1,3,4], 'date')

from ts_toolbox import dp_series_to_supervised
data = data.drop(price_data.columns[price_data.columns != 'Close'],axis = 1)
from sklearn.preprocessing import MinMaxScaler

scaler_x = MinMaxScaler(feature_range=(0, 1))
data.iloc[:,:-1] = scaler_x.fit_transform(data.iloc[:,:-1])

scaler_y = MinMaxScaler(feature_range=(0, 1))
data.iloc[:,-1] = scaler_y.fit_transform(data.iloc[:,-1].to_numpy().reshape(-1,1))

data_lstm = dp_series_to_supervised(data, n_in = 10, n_out =1,y = 'Close')
    

from ts_toolbox import dp_train_test_split
data_train, data_test = dp_train_test_split(data_lstm,train_rate = 0.8)

X_train = data_train.drop('Close(t)',axis=1).to_numpy()
y_train = data_train['Close(t)'].to_numpy()

X_test = data_test.drop('Close(t)',axis=1).to_numpy()
y_test = data_test['Close(t)'].to_numpy()

# data_train.drop('Close(t)',axis=1).to_csv('X_train.csv')
# data_train['Close(t)'].to_csv('y_train.csv')

# data_test.drop('Close(t)',axis=1).to_csv('X_test.csv')
# data_test['Close(t)'].to_csv('y_test.csv')

# Colab

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot
import numpy as np

# X_train = pd.read_csv('X_train.csv',index_col = 0).to_numpy()
# y_train = pd.read_csv('y_train.csv',index_col = 0).to_numpy()
# X_test = pd.read_csv('X_test.csv',index_col = 0).to_numpy()
# y_test = pd.read_csv('y_test.csv',index_col = 0).to_numpy()

# reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(X_train, y_train, epochs=100, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

  # make a prediction
ypred = model.predict(X_test)
y_pred = scaler_y.inverse_transform(ypred)
y_true = scaler_y.inverse_transform(y_test.reshape(-1,1))
y_drift = pd.DataFrame(y_true).shift(1).iloc[:,0]
result = pd.DataFrame(data = {
                       'y_true':y_true.reshape(1,-1)[0],
                       'y_pred':y_pred.reshape(1,-1)[0],
                       'y_drift':y_drift
                       }).reset_index(drop = False).melt(id_vars = ['index'])
fig = px.line(result, x="index", y="value", color='variable')
fig.show()